In [1]:
import numpy as np

In [2]:
a=np.array([[1,2,3],[4,5,6]])
b=np.array([7,8,9])
np.dot(a,b)

array([ 50, 122])

In [ ]:
class FullConnectedLayer:
    def __init__(self, input_dim, output_dim, activator):
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.activator = activator
        self.weight = np.random.uniform(-0.1, 0.1, [self.output_dim, self.input_dim])
        self.bias = np.zeros([self.output_dim, 1])
    def forward(self, input):
        self.output = self.activator(np.dot(self.weight, input)+self.bias)
    def backward(self, delta):
        